In [1]:
# Data files
normal_traffic_file_1 = 'normalTrafficTraining.txt'
normal_traffic_file_2 = 'normalTrafficTest.txt'
anomalous_traffic_file = 'anomalousTrafficTest.txt'

In [2]:
# Load data and remove newlines characters
def load_data(file_name):
    file = open(file_name, 'r')
    contents = file.read().split('\n')
    file.close()
    return contents

In [3]:
# Load normal and anomalous data into seperate lists
normal_traffic = load_data(normal_traffic_file_1) + load_data(normal_traffic_file_2)
anomalous_traffic = load_data(anomalous_traffic_file)

In [4]:
# Create a list of http requests
def create_requests_list(traffic_list):
    traffic_requests = []
    for line_num in range(len(traffic_list)):
        line = traffic_list[line_num]
        
        if line.startswith('GET'):
            # mozna jeste odstranim HTTP/1.1 ci na zacatku localhost
            # Remove unnecessary white spaces and uppercase
            line = line.lower().replace(' ', '')
            traffic_requests.append(line)
            
        elif line.startswith('POST') or line.startswith('PUT'):
            request_str = line
            while not line.startswith('Content-Length:'):
                line_num += 1
                line = traffic_list[line_num]
            # Second line below 'Content-Length' may be relevant
            request_str = request_str + traffic_list[line_num + 2]
            request_str = request_str.lower().replace(' ', '')
            traffic_requests.append(request_str)
    return traffic_requests

In [5]:
normal_traffic_request = create_requests_list(normal_traffic)
anomalous_traffic_request = create_requests_list(anomalous_traffic)

In [6]:
#anomalous_traffic_request
#len(anomalous_traffic_request)